In [1]:
env = "production"
# env = "staging"


# s3_bucket = "s3://mist-aggregated-stats-{env}/entity_event/entity_event-{env}/".format(env=env)
date_day = "2020-03-1*"
hr = '*'

s3_bucket = "s3://mist-secorapp-staging/sle-coverage-anomaly/sle-coverage-anomaly-staging/".format(env=env)
s3_path = s3_bucket + "dt={day}/hr={hr}/*.seq".format(day=date_day, hr=hr)

print(s3_path)


s3://mist-secorapp-staging/sle-coverage-anomaly/sle-coverage-anomaly-staging/dt=2020-03-1*/hr=*/*.seq


In [2]:
# rdd_capacity = spark.sparkContext.sequenceFile(s3_capacity_path)
# rdd_coverage = spark.sparkContext.sequenceFile(s3_coverage_path)
rdd_coverage = spark.sparkContext.sequenceFile(s3_path)
rdd_coverage.first()

(4109,
 bytearray(b'{"radio_5s":1,"util_nonwifi_std_base":10.686252287455485,"channel":11,"sle_coverage_base":0.9739787605262338,"util_nonwifi_deviation":-0.953568937864141,"rssi_mean":-89.0,"rad_id":"r24","sle_capacity_anomaly_score":-0.001005110795669877,"util_all_mean_base":26.527651431022772,"anomaly_score":2.698591450511145,"dev":"r1","anomaly_type":"weak_signal","band":"24","util_cochannel_std":5.0,"sle_coverage_anomaly":true,"util_cochannel_mean":13.068257183736655,"util_cochannel_std_base":5.0,"events":[],"avg_nclients":1.0,"sle_capacity_error":0.348010216963685,"util_nonwifi_mean_base":14.23794877362038,"util_ap":1.971286614351615,"sle_coverage_error":0.3517354443316347,"util_cochannel_deviation":-3.5514748490356416,"ap":"5c-5b-35-0e-b3-ac","rssi_hist":{"-89":180},"error_rate":0.7976085778719924,"sle_coverage":0.0,"user_seconds_util_high":0,"sle_coverage_base_std":0.13839426885618095,"sle_coverage_anomaly_score":2.698591450511145,"interval":60,"impacted_wlans":["c43d016e-8de6-

In [3]:
import json
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"), x.get("ap_id"))).groupByKey()
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"))).groupBy('site_id')
df_coverage = rdd_coverage.map(lambda x: json.loads(x[1])).toDF() #.map(lambda x: json.loads(x[1])).



/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [4]:
# df_capacity
df_coverage.printSchema()

root
 |-- anomaly: string (nullable = true)
 |-- anomaly_score: double (nullable = true)
 |-- anomaly_type: string (nullable = true)
 |-- ap: string (nullable = true)
 |-- avg_nclients: double (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- dev: string (nullable = true)
 |-- error_rate: double (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- impacted_wlans: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- interval: long (nullable = true)
 |-- max_nclients: double (nullable = true)
 |-- org: string (nullable = true)
 |-- power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- radio_5s: long (nullable = true)
 |-- rssi_deviation: double (nullable = true)
 |-- rssi_hist: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- rssi_mean: double (nullable = true

In [5]:

# df_capacity.select("org", "site",  "anomaly", "anomaly_score", "anomaly_type", "sle_capacity_anomaly", "util_nonwifi_mean", "util_cochannel_mean").show()

df_coverage_g = df_coverage.select("org", "site", "anomaly_type").groupBy("org", "site", "anomaly_type").count()

In [7]:
df_coverage_g.orderBy('count', ascending=False).take(10)

# df_capacity_g.sort('count', ascending=False).show() #


[Row(org='6748cfa6-4e12-11e6-9188-0242ac110007', site='67970e46-4e12-11e6-9188-0242ac110007', anomaly_type='weak_signal', count=48),
 Row(org='df6a2fdc-5711-11e5-bd84-1258369c38a9', site='ee915a44-b963-11e5-b14c-1258369c38a9', anomaly_type='weak_signal', count=39),
 Row(org='f4d3653e-4aa3-11e5-8510-1258369c38a9', site='29d2648e-e66b-11e5-a6cd-0242ac110003', anomaly_type='weak_signal', count=25),
 Row(org='60c27646-e75f-11e6-be92-0242ac110003', site='0a6ceeb3-4b49-4df9-a235-14ac9c85b884', anomaly_type='weak_signal', count=18),
 Row(org='19163200-4a90-11e5-8b45-1258369c38a9', site='1916d52a-4a90-11e5-8b45-1258369c38a9', anomaly_type='asymmetry_downlink', count=15),
 Row(org='b0b9f142-aaba-11e6-aafc-0242ac110002', site='fc656275-b157-43fd-b922-5f4f341c19bf', anomaly_type='weak_signal', count=15),
 Row(org='2dbab92c-7cc4-11e5-ab8b-1258369c38a9', site='dbc8209a-bc93-11e5-9c9b-1258369c38a9', anomaly_type='weak_signal', count=11),
 Row(org='b425e4f2-ad72-11e6-8a23-0242ac110004', site='c4a51a2